# Test Generator

Read in the candidates and item data and generate a randomised test from them.

We assume that the 1PL model is used.

$$
Pr(X=1) = \frac{exp(\theta-b)}{1 + exp(\theta-b)}
$$

This test generator can generate tests with a mix of dichotomous and polytomous (scalar) items. The items are loaded from the file `data/items.csv` which looks like this:

```
UIID,a,b,se,rating,k
A1L_7616,1.0,-0.35,0.0,A1,2
A1L_20679,1.0,-0.422,0.0,A1,2
W1,1.0,0.0,0.0,B1+,4
W2,1.0,0.0,0.0,B1+,7
```

Here, the first two items: `A1L_7616` and `A1L_20679` are dichotomous (ie. 2 levels) where `k = 1`, so they can take values '0' (incorrect) or '1' (correct); they have difficulty parameters `b` of -0.35 and -0.422 respectively. There are also two polytomous items `W1` and `W2` which have 4 and 7 levels respectively (`k = 3` and `k = 6`). Currently the difficulty thresholds used for the polytomous items are hardcoded in the `POLY_THRESHOLDS` list (see below) - they should really be different for each item.

## Data Ingest

There are two files in the `data` folder that we need: `items.csv` and `candidates.csv`. From these we generate a randomised test.

In [ ]:
import numpy as np
from numpy.random import seed
from typing import List, Tuple
from csv import reader
import pandas as pd


# these are the threshold params (deltas) used for the scalar items
# in a proper item bank, each item would have its own set of thresholds
POLY_THRESHOLDS = {
        '12': {'1': -5.39, '2': -4.29, '3': -3.23, '4': -2.21, '5': -1.72, '6': -1.23,
              '7': -0.26, '8': 0.72, '9': 1.00, '10': 1.74, '11': 2.8, '12': 3.9},
        '15': {'1': -5.39, '2': -4.90, '3': -4.29, '4': -3.23, '5': -2.87, '6': -2.21,
              '7': -1.23, '8': -0.80, '9': -0.26, '10': 0.72, '11': 1.10, '12': 1.74, '13': 2.8, '14': 3.3, '15': 3.9},
        '18': {'1': -5.39, '2': -5.14, '3': -4.29, '4': -3.70, '5': -3.23, '6': -2.78, '7': -2.21, '8': -1.87,
              '9': -1.23, '10': -0.26, '11': 0.27, '12': 0.72, '13': 0.75, '14': 1.74, '15': 2.1, '16': 2.8, '17': 3.4, '18': 3.9},
        '22': {'1': -5.39, '2': -4.94, '3': -4.51, '4': -4.06, '5': -3.62, '6': -3.17, 
               '7': -2.73, '8': -2.29, '9': -1.85, '10': -1.40, '11': -0.96, '12': -0.52,
               '13': -0.081, '14': 0.36, '15': 0.80, '16': 1.24, '17': 1.68, '18': 2.13,
               '19': 2.57, '20': 3.015, '21': 3.45, '22': 3.9},
        '24': {'1': -5.39, '2': -4.98, '3': -4.58, '4': -4.17, '5': -3.77, '6': -3.37, 
               '7': -2.96, '8': -2.56, '9': -2.15, '10': -1.75, '11': -1.35, '12': -0.94, 
               '13': -0.54, '14': -0.13, '15': 0.26, '16': 0.66, '17': 1.07, '18': 1.47, 
               '19': 1.88, '20': 2.28, '21': 2.68, '22': 3.092, '23': 3.496, '24': 3.9},
        '30': {'1': -5.39, '2': -5.06, '3': -4.74, '4': -4.42, '5': -4.10, 
               '6': -3.78, '7': -3.46, '8': -3.14, '9': -2.82, '10': -2.50,
               '11': -2.18, '12': -1.86, '13': -1.54, '14': -1.36, '15': -1.23, 
               '16': -0.58, '17': -0.34, '18': -0.26, '19': 0.055, '20': 0.37,
               '21': 0.69, '22': 1.016, '23': 1.33, '24': 1.65, '25': 1.97,
               '26': 2.29, '27': 2.61, '28': 2.93, '29': 3.25, '30': 3.9}
}


def getThresholds(k: int):
    assert k in [12, 15, 18, 22, 24, 30]
    
    thresholds = POLY_THRESHOLDS.get(str(k))
    
    return thresholds

    

def getDataAsList(datafile: str) -> List[Tuple]:
    """Turn a CSV datafile into a list of tuples

    :param datafile: the CSV file to load data from
    :return: a list of rows (tuples)
    """
    with open(datafile, 'r', encoding='utf-8-sig') as fs:
        csv_reader = reader(fs)
        row_list = list(map(tuple, csv_reader))
        return row_list[1:]    # ignore the header row
    

# convert the raw data from the candidates.csv file into
# a simple quadruple of ( systemname, givenName, familyName, theta )
def getCandidates() -> List[Tuple]:
    candidates = getDataAsList('data/candidates.csv')
    new_list = [(c[0], c[1], c[2], float(c[3])) for c in candidates]
    return new_list
    

# convert the raw data from the items.csv file into
# a simple triple of ( uiid, a, b )
def getItems() -> List[Tuple]:
    items = getDataAsList('data/items.csv')
    new_list = [(i[0], float(i[1]), float(i[2]), int(i[5])) for i in items]
    return new_list

In [ ]:
items = getItems()
candidates = getCandidates()

## Item Response Generation

The `getItemResponse()` function is used to generate a randomised response: correct (1) or incorrect (0) for a given candidate taking an item.

In [ ]:
def getItemResponse(b: float, theta: float, seed: int = None) -> str:
    """Gets a randomised dichotomous item response for a given candidate
    according to the 1PL model:

    P(X=1) = e^(theta-b) / 1 + e^(theta-b)

    :param b: the difficulty parameter for the item
    :param theta: the latent ability of the candidate
    :return: '0' = incorrect, '1' = correct
    """
    if seed is None:
        rng = np.random.default_rng()
    else:
        rng = np.random.default_rng(seed)
    rv = rng.random()

    p1 = np.exp(theta - b) / (1 + np.exp(theta - b))
    p0 = 1 - p1

    assert p0 <= 1.0
    assert p1 <= 1.0

    rLookup = {
        '0': [0.00, p0],
        '1': [p0, 1.00]
    }
    r = {k: v for (k, v) in rLookup.items() if v[0] <= rv <= v[1]}
    rKey = list(r.keys())

    assert rKey[0] == '1' or rKey[0] == '0'

    return rKey[0]


def getScalarResponse(b: float, theta: float, k: int = 1, seed: int = None) -> str:
    """Gets a randomised polytomous item response for a given candidate
    according to the 1PL model; uses the partial credit model (PCM) approach.
    It uses the thresholds from POLY_THRESHOLDS to calculate what the difficulties
    should be for each level.

    :param b: the difficulty parameter for the item
    :param theta: the latent ability of the candidate
    :param k: the number of levels for the item (1 = dichotomous, 2+ = polytomous)
    :param maxValue: the maximum possible value of the scalar response
    :return: '0'-k
    """
    assert k > 0
    
    response = 0
    
    if k == 1:
        response = getItemResponse(b, theta, seed)
    else:        
        if seed is None:
            rng = np.random.default_rng()
        else:
            rng = np.random.default_rng(seed)
        rv = rng.random()
        
        rLookup = {}
        rThresholds = getThresholds(k)
        rLookup['0'] = [0.00, rThresholds['1']]
        p1 = p0 = 0.00
        
        for t in range(k+1):
            p0 = p1
            if t == k:
                p1 = 1.0
            else:
                b = rThresholds[str(t+1)]
                p1 = 1 - (np.exp(theta - b) / (1 + np.exp(theta - b)))
            rLookup[str(t)] = [p0, p1]

        r = {k: v for (k, v) in rLookup.items() if v[0] <= rv <= v[1]}
        rKey = list(r.keys())    
        response = rKey[0]
        
    return response


We iterate through the data and genereate item responses for each candidate. Each candidate takes a test comprising each item; with a simulated response being generated for each.

In [ ]:
def GenerateRandomTests(seed: int = None):
    test_responses = [] # a list of lists

    # generate a header row for the results
    header = []
    header.append('systemname')
    for i in items:
        header.append(i[0])

    # now create the simulated test responses
    for c in candidates:
        test = []
        test.append(c[0])
        for i in items:
            if i[3] > 2:
                # proivde a polytomous response
                r = int(getScalarResponse(i[2], c[3], i[3], seed))
            else:
                # provide a dichomtomous response
                r = int(getItemResponse(i[2], c[3], seed))
            test.append(r)
        test_responses.append(test)

    df = pd.DataFrame(test_responses, columns=header)
    return df

In [ ]:
df = GenerateRandomTests()

(df)

## Next Steps

You can call the `GenerateRandomTests()` function as many times as you want to re-generate a test. It will generate different results every time (unless you pass in an integer seed value).

Add items and candidates to the data files to generate larger tests.

When you are happy with the results you can write out to a results CSV file like this:

In [ ]:
df.to_csv('data/results.csv', index=False)